# Using ROBOKOP's quick service to find hypoglycemia treatments

The most basic functionality in answering questions is to start with an entity and find other connected entities.
In this context, an entity is defined by a curie-formatted identifier.

Quick handles general pattern matching queries with arbitrarily specified nodes and edges.  In this case, we're going to run this query:

(?)-[treats]->(Hypoglycemia)

And see what ROBOKOP returns.

These cells set up some of the functions that we want to use:

In [1]:
import requests
import json
import pandas as pd

robokop_server = 'robokop.renci.org'

def quick(question):
    """Call robokop's quick service with a json-question"""
    url=f'http://{robokop_server}:80/api/simple/quick/'
    response = requests.post(url,json=question)
    print( f"Return Status: {response.status_code}" )
    if response.status_code == 200:
        return response.json()
    return response

def make_N_step_question(types,curies,props,forwards):
    """Create a json question that can be passed to quick.  The question will be a linear chain where
    the nodes on the chain have types, and each node can have an optional query. There can also be 
    optional predicate types (props).  Forwards is an array with the same cardinality as props and indicates
    the direction of the edge."""
    question = {
                'machine_question': {
                    'nodes': [],
                    'edges': []
                }
            }
    for i,t in enumerate(types):
        newnode = {'id': f'n{i}', 'type': t}
        if curies[i] is not None:
            newnode['curie'] = curies[i]
        question['machine_question']['nodes'].append(newnode)
        if i > 0:
            if forwards[i-1]:
                edge = {'id': f'e{i}', 'source_id': f'n{i-1}', 'target_id': f'n{i}'}
            else:
                edge = {'id': f'e{i}', 'source_id': f'n{i}', 'target_id': f'n{i-1}'}
            if props[i-1] is not None:
                edge['type'] = props[i-1]
            question['machine_question']['edges'].append( edge )
    return question

def answers2frame(graph_answers):
    """A function for taking the standard answer format and converting it into a simple frame.
    Specific to the question in this notebook."""
    answers = []
    for graph_answer in graph_answers['answers']:
        for node in graph_answer['nodes']:
            if node['type'] == 'chemical_substance':
                chemical=node
        ans = { 'score': graph_answer['score'],
                'chem_id': chemical['id'],
                'chem_name': chemical['name']
              }
        answers.append(ans)
    df = pd.DataFrame(answers)
    ordered_columns = ['score','chem_id','chem_name']
    df = df[ordered_columns]
    return df

In [2]:
#Specify the particular question we want to answer.  This says, start at the disease MONDO:0004946 (which is
# hypoglycemia) and find a chemical_substance that treats it.  The [False] in the final argument means that the
# direction of the last edge is 'backwards' from chemical_substance to disease.
q = make_N_step_question(['disease','chemical_substance'], ['MONDO:0004946', None], ['treats'], [False])

#What does the question format look like?
import json
print(json.dumps(q,indent=4))

{
    "machine_question": {
        "nodes": [
            {
                "id": "n0",
                "type": "disease",
                "curie": "MONDO:0004946"
            },
            {
                "id": "n1",
                "type": "chemical_substance"
            }
        ],
        "edges": [
            {
                "id": "e1",
                "source_id": "n1",
                "target_id": "n0",
                "type": "treats"
            }
        ]
    }
}


In [3]:
a = quick(q)

Return Status: 200


In [4]:
df = answers2frame(a)
df

,score,chem_id,chem_name
0,1.740737,CHEBI:17234,glucose
1,1.422864,CHEBI:4495,diazoxide
2,1.147535,CHEBI:5391,glucagon
3,0.694123,CHEBI:17855,triglyceride
4,0.618756,CHEBI:3650,chlorpropamide
5,0.570677,CHEBI:41879,dexamethasone
6,0.510838,CHEBI:15940,nicotinic acid
7,0.510366,CHEBI:3699,cimetidine
8,0.508147,CHEBI:4636,diphenhydramine
9,0.508147,CHEBI:28821,piperine
